# Utilisation d'outils

# Les outils fournis par OpenAI

In [1]:
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

llm_model = "gpt-4o"

In [2]:
from agents import Agent, CodeInterpreterTool, Runner, WebSearchTool

agent = Agent(
    name="Assistant",
    model=llm_model,
    tools=[
        WebSearchTool(),
        CodeInterpreterTool(
            tool_config={"type": "code_interpreter", "container": {"type": "auto"}},
        ),
    ],
)

async def main():
    result = await Runner.run(agent, "What is the current weather in New York City in Fahrenheit? Convert it into Celsius")
    print(result.final_output)

In [3]:
await main()

The current temperature in New York City is 70°F, which is approximately 21°C.


## Utilisation d'outils custom

In [4]:
import json

from typing_extensions import TypedDict, Any

from agents import Agent, FunctionTool, RunContextWrapper, function_tool


class Location(TypedDict):
    lat: float
    long: float

@function_tool  
async def fetch_weather(location: Location) -> str:
    
    """Fetch the weather for a given location.

    Args:
        location: The location to fetch the weather for.
    """
    # In real life, we'd fetch the weather from a weather API
    return "sunny"


@function_tool(name_override="fetch_data")  
def read_file(ctx: RunContextWrapper[Any], path: str, directory: str | None = None) -> str:
    """Read the contents of a file.

    Args:
        path: The path to the file to read.
        directory: The directory to read the file from.
    """
    # In real life, we'd read the file from the file system
    return "<file contents>"


agent = Agent(
    name="Assistant",
    tools=[fetch_weather, read_file],  
)

for tool in agent.tools:
    if isinstance(tool, FunctionTool):
        print(tool.name)
        print(tool.description)
        print(json.dumps(tool.params_json_schema, indent=2))
        print()

fetch_weather
Fetch the weather for a given location.
{
  "$defs": {
    "Location": {
      "properties": {
        "lat": {
          "title": "Lat",
          "type": "number"
        },
        "long": {
          "title": "Long",
          "type": "number"
        }
      },
      "required": [
        "lat",
        "long"
      ],
      "title": "Location",
      "type": "object",
      "additionalProperties": false
    }
  },
  "properties": {
    "location": {
      "description": "The location to fetch the weather for.",
      "properties": {
        "lat": {
          "title": "Lat",
          "type": "number"
        },
        "long": {
          "title": "Long",
          "type": "number"
        }
      },
      "required": [
        "lat",
        "long"
      ],
      "title": "Location",
      "type": "object",
      "additionalProperties": false
    }
  },
  "required": [
    "location"
  ],
  "title": "fetch_weather_args",
  "type": "object",
  "additionalProperties

In [5]:
await Runner.run(agent, "What is the current weather in New York City in Fahrenheit? Convert it into Celsius")

RunResult(input='What is the current weather in New York City in Fahrenheit? Convert it into Celsius', new_items=[ToolCallItem(agent=Agent(name='Assistant', instructions=None, handoff_description=None, handoffs=[], model=None, model_settings=ModelSettings(temperature=None, top_p=None, frequency_penalty=None, presence_penalty=None, tool_choice=None, parallel_tool_calls=None, truncation=None, max_tokens=None, reasoning=None, metadata=None, store=None, include_usage=None, extra_query=None, extra_body=None, extra_headers=None), tools=[FunctionTool(name='fetch_weather', description='Fetch the weather for a given location.', params_json_schema={'$defs': {'Location': {'properties': {'lat': {'title': 'Lat', 'type': 'number'}, 'long': {'title': 'Long', 'type': 'number'}}, 'required': ['lat', 'long'], 'title': 'Location', 'type': 'object', 'additionalProperties': False}}, 'properties': {'location': {'description': 'The location to fetch the weather for.', 'properties': {'lat': {'title': 'Lat', '